In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
import datetime
import time
import matplotlib.pyplot as plt
import wrds
from pandas.tseries.offsets import *
import pandas_datareader

d:\Applications\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Load Data

In [2]:
conn = wrds.Connection(wrds_username = "frankgong")
crsp_raw = conn.raw_sql("""
                        select a.permno, a.permco, a.date, b.shrcd, b.exchcd, 
                      a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr
                      from crspq.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1926' and '12/31/2024'
                        """)
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop = True).copy()
crsp_raw[['permno','permco']] = crsp_raw[['permno','permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'],format = "%Y-%m-%d",errors = 'ignore')+MonthEnd(0)



Loading library list...
Done


C:\Users\14561\AppData\Local\Temp\ipykernel_29144\805863334.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  crsp_raw['date'] = pd.to_datetime(crsp_raw['date'],format = "%Y-%m-%d",errors = 'ignore')+MonthEnd(0)


In [3]:
crsp_raw


,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc,cfacshr,cfacpr
0,10000,7952,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,7952,1986-01-31,10.0,3.0,NaN,NaN,3680.0,-4.37500,1.0,1.0
2,10000,7952,1986-02-28,10.0,3.0,-0.257143,-0.257143,3680.0,-3.25000,1.0,1.0
3,10000,7952,1986-03-31,10.0,3.0,0.365385,0.365385,3680.0,-4.43750,1.0,1.0
4,10000,7952,1986-04-30,10.0,3.0,-0.098592,-0.098592,3793.0,-4.00000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
5036828,93436,53453,2023-08-31,11.0,3.0,-0.034962,-0.034962,3173994.0,258.07999,1.0,1.0
5036829,93436,53453,2023-09-30,11.0,3.0,-0.030456,-0.030456,3179000.0,250.22000,1.0,1.0
5036830,93436,53453,2023-10-31,11.0,3.0,-0.197346,-0.197346,3178921.0,200.84000,1.0,1.0
5036831,93436,53453,2023-11-30,11.0,3.0,0.195379,0.195379,3178921.0,240.08000,1.0,1.0


In [3]:
dlret_raw = conn.raw_sql("""
                         select permno, dlret, dlstdt, dlstcd
                         from crspq.msedelist
                         """)
dlret_raw = dlret_raw.sort_values(['permno','dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date'] = dlret_raw['dlstdt']+MonthEnd(0)

In [4]:
FF3 = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors', start = '1900',end = str(datetime.datetime.now().year+1))
FF3 = FF3.read()[0]/100
FF3.columns = 'MktRF','SMB','HML','RF'
FF3['Mkt'] = FF3['MktRF']+FF3['RF']
FF3 = FF3.reset_index().rename(columns = {"Date":"date"}).copy()
FF3['date'] = pd.DataFrame(FF3["date"].values.astype('datetime64[ns]'))+MonthEnd(0)


C:\Users\14561\AppData\Local\Temp\ipykernel_29144\2418989498.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  FF3 = FF3.read()[0]/100
C:\Users\14561\AppData\Local\Temp\ipykernel_29144\2418989498.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  FF3 = FF3.read()[0]/100


In [5]:
FF3

,date,MktRF,SMB,HML,RF,Mkt
0,1926-07-31,0.0296,-0.0256,-0.0243,0.0022,0.0318
1,1926-08-31,0.0264,-0.0117,0.0382,0.0025,0.0289
2,1926-09-30,0.0036,-0.0140,0.0013,0.0023,0.0059
3,1926-10-31,-0.0324,-0.0009,0.0070,0.0032,-0.0292
4,1926-11-30,0.0253,-0.0010,-0.0051,0.0031,0.0284
...,...,...,...,...,...,...
1167,2023-10-31,-0.0319,-0.0387,0.0019,0.0047,-0.0272
1168,2023-11-30,0.0884,-0.0002,0.0164,0.0044,0.0928
1169,2023-12-31,0.0487,0.0634,0.0493,0.0043,0.0530
1170,2024-01-31,0.0071,-0.0509,-0.0238,0.0047,0.0118


In [6]:
# merge dlret into ret
crsp = crsp_raw.merge(dlret_raw[['permno','dlret','date']], on=['permno','date'],how = 'outer')
crsp  = crsp[['permno','date','shrcd','exchcd','ret','dlret','prc','shrout']]
crsp

,permno,date,shrcd,exchcd,ret,dlret,prc,shrout
0,10000,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-01-31,10.0,3.0,NaN,NaN,-4.37500,3680.0
2,10000,1986-02-28,10.0,3.0,-0.257143,NaN,-3.25000,3680.0
3,10000,1986-03-31,10.0,3.0,0.365385,NaN,-4.43750,3680.0
4,10000,1986-04-30,10.0,3.0,-0.098592,NaN,-4.00000,3793.0
...,...,...,...,...,...,...,...,...
5036828,93436,2023-08-31,11.0,3.0,-0.034962,NaN,258.07999,3173994.0
5036829,93436,2023-09-30,11.0,3.0,-0.030456,NaN,250.22000,3179000.0
5036830,93436,2023-10-31,11.0,3.0,-0.197346,NaN,200.84000,3178921.0
5036831,93436,2023-11-30,11.0,3.0,0.195379,NaN,240.08000,3178921.0


#### Question 1

In [7]:
def PS1_Q1(crsp):
    # control exchcd and shrcd
    # lets keep NYSE NASDAQ (1  and 3 )
    # Also only keep 10: Ordinary common shares. 11: Common shares with different voting rights.
    
    crsp = crsp[(crsp['exchcd'].isin([1,2,3]) & crsp['shrcd'].isin([10,11]))]
    
    # merge dlret and fix ret
    # if ret and dlret both exist
    crsp['ret'] = np.where(crsp['ret'].notna() & crsp['dlret'].notna(),(1+crsp['ret'])*(1+crsp['dlret'])-1, crsp['ret'])
    # if only one value exist
    crsp['ret'] = np.where(crsp['ret'].isna()  & crsp['dlret'].notna(), crsp['dlret'], crsp['ret'])
    # if both na
    crsp['ret'] = np.where(crsp['ret'].isna() & crsp['dlret'].isna(),0,crsp['ret'])
    
    # market equity
    crsp['market equity'] = abs(crsp['prc'])*crsp['shrout']
    
    # lag market equity
    crsp['lag market equity'] = crsp[['permno','market equity']].groupby('permno').shift(1) 
    
    # equal weighted return
    ew_ret = crsp.groupby('date')['ret'].mean()
    
    #lag MV
    lag_mv = crsp['lag market equity'].groupby(crsp['date']).sum()/1e6
    
    # value weighted return
    crsp['vw_ret'] = crsp['lag market equity']*crsp['ret']
    vw_ret = crsp['vw_ret'].groupby(crsp['date']).sum()/crsp['lag market equity'].groupby(crsp['date']).sum()
    

    
    return_df = pd.DataFrame({
        "year":ew_ret.index.year,
        "month":ew_ret.index.month,
        "Stock_lag_MV":lag_mv,
        "Stock_Ew_Ret":ew_ret,
        "Stock_Vw_Ret":vw_ret
    })
    # return
    return return_df
    
Monthly_CRSP_Stocks = PS1_Q1(crsp)
Monthly_CRSP_Stocks



C:\Users\14561\AppData\Local\Temp\ipykernel_29144\1927133385.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['ret'] = np.where(crsp['ret'].notna() & crsp['dlret'].notna(),(1+crsp['ret'])*(1+crsp['dlret'])-1, crsp['ret'])
C:\Users\14561\AppData\Local\Temp\ipykernel_29144\1927133385.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['ret'] = np.where(crsp['ret'].isna()  & crsp['dlret'].notna(), crsp['dlret'], crsp['ret'])
C:\Users\14561\AppData\Local\Temp\ipykernel_29144\1927133385.py:14: Set

,year,month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret
date,,,,,
1926-01-31,1926,1,0.000000,0.022343,NaN
1926-02-28,1926,2,27.032250,-0.051437,-0.033976
1926-03-31,1926,3,26.128629,-0.093698,-0.064875
1926-04-30,1926,4,24.506932,0.030374,0.036735
1926-05-31,1926,5,25.178699,0.000875,0.011971
...,...,...,...,...,...
2023-08-31,2023,8,45247.097359,-0.068190,-0.019338
2023-09-30,2023,9,44202.187878,-0.069618,-0.048173
2023-10-31,2023,10,41958.286962,-0.078985,-0.027157


In [8]:
Monthly_CRSP_Stocks.to_csv("Stock_CRSP.csv")

#### Question 2

In [51]:


#FF3["Market_minus_Rf"] = FF3["MktRF"]-FF3["RF"]
FF3["date"] == pd.to_datetime(FF3["date"])
FF3["year"] = FF3['date'].dt.year
FF3["month"] = FF3['date'].dt.month
#FF3_2 = FF3[["year","month","Market_minus_Rf","SMB","HML","RF"]]
FF3_2 = FF3[["year","month",'MktRF',"SMB","HML","RF"]]
FF_mkt = FF3_2[:-2]
FF_mkt['RF']  = pd.to_numeric(FF_mkt['RF'])


C:\Users\14561\AppData\Local\Temp\ipykernel_19660\1538663411.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FF_mkt['RF']  = pd.to_numeric(FF_mkt['RF'])


In [49]:

Monthly_CRSP_Stocks_FF = pd.merge(Monthly_CRSP_Stocks,FF_mkt, on=['year','month'], how='outer')
Monthly_CRSP_Stocks_FF.dropna(inplace=True)
Monthly_CRSP_Stocks_FF['Ex_ret'] = Monthly_CRSP_Stocks_FF["Stock_Vw_Ret"]-Monthly_CRSP_Stocks_FF['RF']
Monthly_CRSP_Stocks_FF

,year,month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret,MktRF,SMB,HML,RF,Ex_ret
6,1926,7,26.699967,0.012027,0.031514,0.0296,-0.0256,-0.0243,0.0022,0.029314
7,1926,8,27.402107,0.029607,0.028824,0.0264,-0.0117,0.0382,0.0025,0.026324
8,1926,9,28.348789,-0.006419,0.005848,0.0036,-0.0140,0.0013,0.0023,0.003548
9,1926,10,28.308142,-0.033157,-0.029081,-0.0324,-0.0009,0.0070,0.0032,-0.032281
10,1926,11,27.501095,0.023788,0.028186,0.0253,-0.0010,-0.0051,0.0031,0.025086
...,...,...,...,...,...,...,...,...,...,...
1171,2023,8,45247.097359,-0.068190,-0.019338,-0.0239,-0.0316,-0.0106,0.0045,-0.023838
1172,2023,9,44202.187878,-0.069618,-0.048173,-0.0524,-0.0251,0.0152,0.0043,-0.052473
1173,2023,10,41958.286962,-0.078985,-0.027157,-0.0319,-0.0387,0.0019,0.0047,-0.031857
1174,2023,11,40654.841180,0.084557,0.092789,0.0884,-0.0002,0.0164,0.0044,0.088389


In [56]:
def PS1_Q2(d1):
    # From FF
    ret_1 = d1["MktRF"]
    # From CRSP
    ret_2 = d1["Ex_ret"]
    Ann_Mean = [np.mean(ret_1)*12,np.mean(ret_2)*12]
    Ann_Std = [np.std(ret_1)*np.sqrt(12),np.std(ret_2)*np.sqrt(12)]
    Ann_SR = [np.mean(ret_1) / np.std(ret_1) * np.sqrt(12), np.mean(ret_2) / np.std(ret_2) * np.sqrt(12)]
    Skew = [ret_1.skew(),ret_2.skew()]
    Ex_Kur = [ret_1.kurtosis(),ret_2.kurtosis()]
    
    return_df = pd.DataFrame({
        'Annual Mean': np.round(Ann_Mean,2),
        'Annual Std': np.round(Ann_Std,2),
        'Annual SR': np.round(Ann_SR,2),
        'Skewness': np.round(Skew,2),
        'Excess Kurtosis': np.round(Ex_Kur,2)
    }, index = ['From FF','From CRSP'])
    return return_df.T
df = PS1_Q2(Monthly_CRSP_Stocks_FF)
df


,From FF,From CRSP
Annual Mean,0.08,0.08
Annual Std,0.19,0.18
Annual SR,0.44,0.44
Skewness,0.16,0.16
Excess Kurtosis,7.41,7.38


#### Question 3

In [62]:
def PS1_Q3(d1):
    # From FF
    ret_1 = d1["MktRF"]
    # From CRSP
    ret_2 = d1["Ex_ret"]
    
    corr = ret_1.corr(ret_2)
    print("The correlation is ",np.round(corr,8))
    
    max_diff = max(abs(ret_1-ret_2))
    print("The maximun difference is ",np.round(max_diff,8))
    
    return [np.round(corr,8),np.round(max_diff,8)]
    
PS1_Q3(Monthly_CRSP_Stocks_FF)

The correlation is  0.99999681
The maximun difference is  0.0022429


[0.99999681, 0.0022429]